In [8]:
import pandas as pd
import numpy as np
import warnings

# --- CONFIGURACIÓN ---
warnings.filterwarnings("ignore", category=UserWarning, module='pandas')

# 1. CARGA DE DATOS
df = pd.read_csv(r"Datos\Originales\cancellation_data_for_mondragon_unibertsitatea_2024.csv")

# 2. LIMPIEZA DE COLUMNAS
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w_]", "", regex=True)
)

# 3. ELIMINAR DUPLICADOS
df = df.drop_duplicates()

# ---------------------------------------------------------
# 4. FILTROS Y CONTEOS (RAW DATA)
# ---------------------------------------------------------

# A) Eliminar valores menores a 0 en lead_time
if "lead_time" in df.columns:
    df = df[df["lead_time"] >= 0]

# B) Gestionar lenght_of_stay
col_stay = "lenght_of_stay" if "lenght_of_stay" in df.columns else "length_of_stay"

if col_stay in df.columns:
    # 1. Contar 0s
    count_stay_0 = (df[col_stay] == 0).sum()
    print(f"Instancias con {col_stay} == 0 encontradas: {count_stay_0}")
    
    # 2. MOSTRAR ESOS CASOS (para observar)
    casos_estancia_0 = df[df[col_stay] == 0]
    if not casos_estancia_0.empty:
        print(f"\n--- Muestra de {col_stay} == 0 (se van a eliminar): ---")
        print(casos_estancia_0.head()) # Muestra las primeras filas
    
    # 3. ELIMINAR LOS CASOS == 0 DEL DF ORIGINAL
    df = df[df[col_stay] != 0]
    print("-> Se han eliminado las filas con estancia igual a 0.")

    # 4. Eliminar estancias superiores a 30 días
    df = df[df[col_stay] <= 30]
    print(f"-> Se han filtrado las filas con {col_stay} > 30.")

# C) Gestionar adult_count
if "adult_count" in df.columns:
    count_adult_0 = (df["adult_count"] == 0).sum()
    print(f"Instancias con adult_count == 0 encontradas: {count_adult_0}")
    
    # --- NUEVO: CREAR DF Y MOSTRAR PARA OBSERVAR ---
    casos_adultos_0 = df[df["adult_count"] == 0].copy()
    
    if not casos_adultos_0.empty:
        print("\n--- Casos con adult_count == 0 (DataFrame creado para observar): ---")
        print(casos_adultos_0)
        print("--------------------------------------------------------------------\n")
    
    # df = df[df["adult_count"] != 0]

# ---------------------------------------------------------
# 5. CONVERSIÓN DE FECHAS
# ---------------------------------------------------------
columnas_fecha = [
    "booked_at", 
    "checkin_time", 
    "checkout_time", 
    "cancelled_at", 
    "asset_opening_date"
]

for col in columnas_fecha:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# ---------------------------------------------------------
# 6. MAPEOS Y VARIABLES CATEGÓRICAS
# ---------------------------------------------------------
month_map = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

day_map = {
    "Monday": 1, "Tuesday": 2, "Wednesday": 3,
    "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7
}

if "checkin_month" in df.columns:
    df["checkin_month"] = df["checkin_month"].map(month_map)

if "checkin_day" in df.columns:
    df["checkin_day"] = df["checkin_day"].map(day_map)

categoricas = [
    "channel", "reservation_status", "room_type", "payment_method",
    "property_name", "country"
]

for col in categoricas:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().str.strip()

# ---------------------------------------------------------
# 7. INGENIERÍA DE VARIABLES
# ---------------------------------------------------------
df["is_cancelled"] = df["cancelled_at"].notna().astype(int)

df["fecha_incoherente"] = False
df.loc[df["checkout_time"] < df["checkin_time"], "fecha_incoherente"] = True
df.loc[df["cancelled_at"] < df["booked_at"], "fecha_incoherente"] = True

df = df[df["fecha_incoherente"] == False]

df["days_before_checkin"] = (
    df["checkin_time"] - df["booked_at"]
).where(df["checkin_time"].notna() & df["booked_at"].notna()).dt.days

df["days_before_cancel"] = (
    df["cancelled_at"] - df["booked_at"]
).dt.days

df["stay_length"] = (
    df["checkout_time"] - df["checkin_time"]
).dt.days

# ---------------------------------------------------------
# 8. FILTROS FINALES (Outliers y Limpieza)
# ---------------------------------------------------------
# Filtrar outliers de days_before_checkin (-1 a 450 días)
df = df.loc[(df["days_before_checkin"] <= 450) & (df["days_before_checkin"] >= -1)].copy()

# Eliminar columna auxiliar
df = df.drop(columns=["fecha_incoherente"])



Instancias con lenght_of_stay == 0 encontradas: 1

--- Muestra de lenght_of_stay == 0 (se van a eliminar): ---
                 booked_at         checkin_time        checkout_time  \
23009  Oct 27, 2023, 01:46  Oct 27, 2023, 04:07  Oct 27, 2023, 11:00   

       lead_time  lenght_of_stay checkin_month checkin_day  adult_count  \
23009          0               0       October      Friday            2   

       child_count     origin  ... recurrence libere_community  \
23009            0  telephone  ...          1              yes   

      bought_products product_count reservation_net_value total_adr  \
23009              no             0                     0       NaN   

            status  cancelled_at cancellation_reason cancellation_lead_time  
23009  checked_out           NaN                 NaN                    NaN  

[1 rows x 36 columns]
-> Se han eliminado las filas con estancia igual a 0.
-> Se han filtrado las filas con lenght_of_stay > 30.
Instancias con adult_count == 

C:\Users\danie\AppData\Local\Temp\ipykernel_11844\1243894190.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\danie\AppData\Local\Temp\ipykernel_11844\1243894190.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\danie\AppData\Local\Temp\ipykernel_11844\1243894190.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")
C:\Users\danie\AppData\Local\Temp\ipykernel_11844\1243894190.py:80: UserWarning: Coul

In [9]:
ruta_destino = 'Datos/Transformados/Limpios/df_limpio.csv'

df.to_csv(ruta_destino, index=False)